# ALIMENTAÇÃO DO DYNAMODB

Aqui varremos os JSONs gerados pelos _notebooks_ anteriores e enviamos ao DynamoDB

In [1]:
import os
from json import loads
import boto3
import decimal
from pprint import pprint
from datetime import datetime

def normalize(target):
    if isinstance(target, str):
        if target:
            return {'S': target}
        else:
            return {'NULL': True}
    elif isinstance(target, bool):
        return {'BOOL': target}
    elif isinstance(target, int) or isinstance(target, decimal.Decimal):
        return {'N': str(target)}
    elif isinstance(target, dict):
        for key, value in target.items():
            target[key] = normalize(value)
        return {'M': target}
    elif isinstance(target, list):
        return {'L': [normalize(each) for each in target]}
    elif target is None:
        return {'NULL': True}
    else:
        raise Exception(f'Tipo inesperado: {target.__class__.__name__}')
        

SESSION = boto3.session.Session(region_name='us-east-1', profile_name='sofie')
DYNAMODB = SESSION.client('dynamodb')

WORK_DIR = os.path.join(os.getcwd(), 'FINAL_RESULT')

CNPJs = set()
CNPJsREPETIDOS = set()

dt_start = datetime.now()

for idx, file_name in enumerate([os.path.join(WORK_DIR, each) for each in os.listdir(WORK_DIR)]):
    with open(file_name, 'rt') as file_handler:
        body = loads(file_handler.read(), parse_float=decimal.Decimal)
        DYNAMODB.delete_item(
            TableName='table_micro_task_in_person',
            Key={'task_id': {'S': body['task_id']}}
        )
        
        if body['receita_ws']['cnpj'] in CNPJs:
            body['status']['state'] = 'REJECTED'
            body['status']['status'] = 'REJECTED'
            body['status']['reason'] = 'CNPJ duplicado'
            CNPJsREPETIDOS.add(body['receita_ws']['cnpj'])
        
        CNPJs.add(body['receita_ws']['cnpj'])
        
        data = {key: normalize(value) for key, value in body.items()}
        
    response = DYNAMODB.put_item(TableName='table_micro_task_in_person', Item=data)
    
    print(f'{idx + 1:>4} - task_id [{data["task_id"]["S"]}] inserido com sucesso!')
          
dt_finish = datetime.now()
          
print(f'*** Foram inseridos {idx + 1} registros ***')
print(f'Tempo: {(dt_finish - dt_start).seconds} segundos')
print('\n')
print(f'{len(CNPJsREPETIDOS)} CNPJs repetidos')


   1 - task_id [388e6d4f-4c44-40f9-a6e3-57795bd9bd76] inserido com sucesso!
   2 - task_id [7e7087a1-d83c-4006-bac3-e63c7e80db88] inserido com sucesso!
   3 - task_id [8e6a25f4-b385-431a-8e7f-e7a63d002a8c] inserido com sucesso!
   4 - task_id [594137af-e592-4f30-9a74-8acb929e9dfb] inserido com sucesso!
   5 - task_id [8df73932-e677-4f06-9c70-80ea6766e011] inserido com sucesso!
   6 - task_id [50013d6d-eedb-4695-8c22-9527bd3e6831] inserido com sucesso!
   7 - task_id [de4dbbf0-b8ad-420a-86ab-aeebd4bbeaae] inserido com sucesso!
   8 - task_id [d198dddf-4c8d-4efd-b3f4-43eb5535480b] inserido com sucesso!
   9 - task_id [cd1dbda5-c580-45c9-870e-13924ffad00b] inserido com sucesso!
  10 - task_id [413bd7cb-54e1-4bcb-9f57-793fde9585ab] inserido com sucesso!
  11 - task_id [51a3631f-d123-4e12-aac4-8708790c5b7c] inserido com sucesso!
  12 - task_id [4005b7b5-5c84-4da0-b1fd-f52086b5d7d0] inserido com sucesso!
  13 - task_id [d5305fa4-2ee7-4c93-a8a5-4a90e7ec7a35] inserido com sucesso!
  14 - task_

 109 - task_id [da04d856-e953-4634-b9fb-b1343381b84d] inserido com sucesso!
 110 - task_id [ad99066c-bd5b-444d-8218-a31b0a102fee] inserido com sucesso!
 111 - task_id [6291f513-3e88-48ae-adc8-e2c9cd424016] inserido com sucesso!
 112 - task_id [f0bea26a-e7b6-469b-89a8-6e843dfdc8d6] inserido com sucesso!
 113 - task_id [9968f157-c215-41ae-b3bd-a07b27788268] inserido com sucesso!
 114 - task_id [9d869e5e-3cb7-420b-b6ed-f6ea32061bbe] inserido com sucesso!
 115 - task_id [edb7564f-d06c-4693-8d91-2e777c0564d7] inserido com sucesso!
 116 - task_id [0961ea10-0714-44a3-a445-a9076ff7ad45] inserido com sucesso!
 117 - task_id [3822660e-2324-47f0-ae37-60910a7c176c] inserido com sucesso!
 118 - task_id [4f65270b-4451-44f7-8d9a-736db556063a] inserido com sucesso!
 119 - task_id [ae9450d0-610c-41d1-8c7e-5fd7bf412db0] inserido com sucesso!
 120 - task_id [4c8fbee8-f950-4490-a49b-abdf11fd4cb0] inserido com sucesso!
 121 - task_id [6a586786-e4d6-4ed4-ba64-fe3b174a9181] inserido com sucesso!
 122 - task_

 217 - task_id [801bdd36-c332-4116-8e60-9e97bb914377] inserido com sucesso!
 218 - task_id [8336790f-952d-491a-84cf-14be1153334f] inserido com sucesso!
 219 - task_id [92d47129-1df8-468a-8e1f-8bad625167c3] inserido com sucesso!
 220 - task_id [6a54db1d-8a35-4739-b750-7deb4e85570e] inserido com sucesso!
 221 - task_id [f1d5e16f-90bf-465d-92df-6136bcdfaca5] inserido com sucesso!
 222 - task_id [2dcf83ee-7c89-4bec-adb2-30c3dbeb9c29] inserido com sucesso!
 223 - task_id [4f9f4ecd-627a-4439-bafb-bcacc6830cb3] inserido com sucesso!
 224 - task_id [e6eb848f-d7d8-44d4-8423-bcf61f1ecaba] inserido com sucesso!
 225 - task_id [a127df56-0622-4511-b445-e8759202b94a] inserido com sucesso!
 226 - task_id [da78b5f8-a569-4a55-b7d6-384a7cc067ed] inserido com sucesso!
 227 - task_id [77aecefb-a1b7-4453-9c9e-826bd6712387] inserido com sucesso!
 228 - task_id [833d24c6-0db7-4cf2-90e6-8326c8d223e8] inserido com sucesso!
 229 - task_id [de620f80-a788-4b9d-ac83-45f4f519849f] inserido com sucesso!
 230 - task_

 325 - task_id [08fc3a30-3090-45a9-a319-2529eceb53ed] inserido com sucesso!
 326 - task_id [1714fd30-3a44-4d6e-a92a-154ceae11c33] inserido com sucesso!
 327 - task_id [faa23bcb-93a5-4e93-831f-045c18e14521] inserido com sucesso!
 328 - task_id [3a657944-ac2d-43d2-8707-c22c55eb2e7a] inserido com sucesso!
 329 - task_id [b708758c-05e5-4cea-b872-3478fc95641a] inserido com sucesso!
 330 - task_id [70ca0fef-0974-4b49-bae4-1a224c5d8bcd] inserido com sucesso!
 331 - task_id [143589a4-0757-42e2-932e-ecce79fda7da] inserido com sucesso!
 332 - task_id [c755ab66-dff6-40dc-9d4f-be6b45f48f2b] inserido com sucesso!
 333 - task_id [27349339-4832-4f70-9431-f81dbb74385d] inserido com sucesso!
 334 - task_id [d26262f8-4201-4cf9-8671-323a90a493a5] inserido com sucesso!
 335 - task_id [b98b5172-2ff0-42f0-9ade-0fe96954d44e] inserido com sucesso!
 336 - task_id [45812e1c-e74c-44f5-a0a6-96f2802b49e0] inserido com sucesso!
 337 - task_id [8b97c53f-d9eb-4c1c-85dd-668fb60ec3e4] inserido com sucesso!
 338 - task_

 433 - task_id [96809f06-6c3b-428b-a0da-1f4f28620e2f] inserido com sucesso!
 434 - task_id [416f9fc9-624a-4d71-a4bc-4f34433dad03] inserido com sucesso!
 435 - task_id [c42082e8-26ad-4d5b-bcae-e767c5111b06] inserido com sucesso!
 436 - task_id [3a2ef4d2-f27a-4788-ac61-c9fd6792e712] inserido com sucesso!
 437 - task_id [fcdfb7a1-eb48-4ec5-8e88-087d09823612] inserido com sucesso!
 438 - task_id [1163fff2-16d5-455c-afa1-bcafad9f5a30] inserido com sucesso!
 439 - task_id [7acd81c1-e12f-4f88-8079-0ca0cc7f451e] inserido com sucesso!
 440 - task_id [eee45556-80c3-42c9-9a4d-b7a33ce8731a] inserido com sucesso!
 441 - task_id [b6fd3bd5-fffc-467f-ac2b-3dffa71fb2d2] inserido com sucesso!
 442 - task_id [1b253094-4ef3-4a40-a209-95145dc731c7] inserido com sucesso!
 443 - task_id [204febd4-2d62-4d71-9a37-a37d3d82f31e] inserido com sucesso!
 444 - task_id [13c459df-6574-413e-aaf8-14bf524a63d9] inserido com sucesso!
 445 - task_id [b4649be0-e759-4a5e-b89a-546f3744163b] inserido com sucesso!
 446 - task_

 541 - task_id [6fabb5c1-3ef4-4f10-9325-e9b4f191a966] inserido com sucesso!
 542 - task_id [9cbed607-ef0b-4b90-8f06-aab22ca21963] inserido com sucesso!
 543 - task_id [987191b5-a65b-4055-bb1b-d40e17b7d74b] inserido com sucesso!
 544 - task_id [1eff2748-95d4-4007-9012-e33e3e5f80da] inserido com sucesso!
 545 - task_id [3085e05e-d6e3-40a6-9821-c00c58260d93] inserido com sucesso!
 546 - task_id [ec37f677-1f2c-4888-9746-649a71e5cd35] inserido com sucesso!
 547 - task_id [fcf5271d-4298-41a2-9ffc-2c302caa52a8] inserido com sucesso!
 548 - task_id [1b069fda-631f-4f61-941b-72f15504dfb8] inserido com sucesso!
 549 - task_id [5de116ce-2227-425f-a3ff-a834955f1632] inserido com sucesso!
 550 - task_id [d3b683ec-27fd-40dc-a3f4-37b385aa46a3] inserido com sucesso!
 551 - task_id [85f41b89-0570-4659-a5f3-c44cc26ac47f] inserido com sucesso!
 552 - task_id [569f682b-cfc9-43ea-91ab-fe943f6a0d7e] inserido com sucesso!
 553 - task_id [bbc6e1be-2eca-43ea-b779-85acbf8d67f1] inserido com sucesso!
 554 - task_

 649 - task_id [32870540-1818-4c58-a12b-d0666bf2a130] inserido com sucesso!
 650 - task_id [0412bcc5-b55e-4ae9-9ff6-c73e2a5f9e90] inserido com sucesso!
 651 - task_id [c750e1fe-4918-4f68-93ae-0c4c440c4aa1] inserido com sucesso!
 652 - task_id [a6afd604-cbf3-406c-8dc3-f04b4ee8f765] inserido com sucesso!
 653 - task_id [f3105ffe-3f1f-4050-8aa2-e582273c93fb] inserido com sucesso!
 654 - task_id [266fdd5b-1554-4b49-8901-f3d114a03e38] inserido com sucesso!
 655 - task_id [3536644a-8d34-42f5-844a-d938c37b85c0] inserido com sucesso!
 656 - task_id [ddd21610-26f4-4a6c-a58d-5a89778696c3] inserido com sucesso!
 657 - task_id [286deae8-728f-4d80-8319-7ca389a3ad93] inserido com sucesso!
 658 - task_id [67f80b93-43a4-4502-9387-1a8fbe47403b] inserido com sucesso!
 659 - task_id [9be86ddf-1d0e-4970-bc98-c4a5dfe00551] inserido com sucesso!
 660 - task_id [b6b24823-e681-46a9-90ce-a696b4b4e3a2] inserido com sucesso!
 661 - task_id [0c52e352-8e15-47b3-9c4a-f888ace6c7a7] inserido com sucesso!
 662 - task_

 757 - task_id [92adda9f-68fa-44fe-a3fb-6cbd1aa9f640] inserido com sucesso!
 758 - task_id [696ec22b-1047-4169-b7cf-cf1dac6b5ccf] inserido com sucesso!
 759 - task_id [f718e6b8-d4c0-4353-96b6-3b8d0ba8b691] inserido com sucesso!
 760 - task_id [03806afa-fdcf-40b9-ae4d-ac14d67934f0] inserido com sucesso!
 761 - task_id [b8708a61-02f1-42a3-9a5a-b45fdfcf523e] inserido com sucesso!
 762 - task_id [4dfd37a9-b504-4761-9b59-2572f5460c2b] inserido com sucesso!
 763 - task_id [822dd1aa-26aa-47f4-b7cf-b044c3ccc48d] inserido com sucesso!
 764 - task_id [3ac7870c-49cb-4239-8fe4-6b4b8c3ba41b] inserido com sucesso!
 765 - task_id [0d1261ab-d62f-4900-aecc-ce27798b4c67] inserido com sucesso!
 766 - task_id [363bcc5d-1118-4f0b-8699-13a3771244ee] inserido com sucesso!
 767 - task_id [87ebf2d8-3da1-49fa-981c-2b505044ec22] inserido com sucesso!
 768 - task_id [e5cfc993-92e1-473e-a636-d8d7822a7125] inserido com sucesso!
 769 - task_id [07e66bff-5727-4adb-813f-2dd50948e395] inserido com sucesso!
 770 - task_

 865 - task_id [e8447394-1acb-4476-8e63-9c27b4aeb717] inserido com sucesso!
 866 - task_id [8f7cdfd5-818f-4e2e-8b7f-76f0da9d4581] inserido com sucesso!
 867 - task_id [3f31b950-d577-4bc2-b8b8-8a7a161154e2] inserido com sucesso!
 868 - task_id [f648aece-6cd6-4ab2-a8c2-0665275f0dfd] inserido com sucesso!
 869 - task_id [7993c8f2-2232-4a5a-bac1-9e0a6cb8548a] inserido com sucesso!
 870 - task_id [a6b1f6c0-9aa3-47a6-a2dd-1415fbd054fa] inserido com sucesso!
 871 - task_id [abf8df67-2e2c-48c1-bc5c-98f6b4a43179] inserido com sucesso!
 872 - task_id [89342145-4678-47ed-bb3b-571bd61a50cf] inserido com sucesso!
 873 - task_id [d9fc48d2-e0e2-43da-b878-263b0d96d685] inserido com sucesso!
 874 - task_id [9b7b5822-ba13-4864-b245-d892a5ff58bb] inserido com sucesso!
 875 - task_id [df25f761-320a-4f8c-85de-90b2b4d94abc] inserido com sucesso!
 876 - task_id [124b6398-4732-4248-8268-4078cdf3c0df] inserido com sucesso!
 877 - task_id [a488f6ec-a454-45c1-91e4-1a61aef10ed0] inserido com sucesso!
 878 - task_

 973 - task_id [27b837f1-78af-4052-8d64-90bfefc6a212] inserido com sucesso!
 974 - task_id [8396e008-5fa1-407d-bf28-7f5c180d3ef9] inserido com sucesso!
 975 - task_id [c208dfd0-e0db-4677-aca6-87962dd09209] inserido com sucesso!
 976 - task_id [484f1f98-d7f0-4a8f-a72d-fff685d5251c] inserido com sucesso!
 977 - task_id [21466056-825a-435f-922e-acf19e4424c7] inserido com sucesso!
 978 - task_id [f3eedb89-9d90-47ab-932a-e0f32bd176da] inserido com sucesso!
 979 - task_id [8a003add-fda5-43cb-a11b-35b46bb77dbb] inserido com sucesso!
 980 - task_id [4b0d3361-d569-4fe9-97c7-67b40fed8495] inserido com sucesso!
 981 - task_id [9c1df974-ef14-43fa-a240-a16bdb631ab4] inserido com sucesso!
 982 - task_id [690f0326-6bce-4176-a58f-6c1da5c6687c] inserido com sucesso!
 983 - task_id [82829344-3e0a-4663-8f01-5f9d4951f22d] inserido com sucesso!
 984 - task_id [ce8f4498-2045-4c04-b4a3-8a4d568c8c93] inserido com sucesso!
 985 - task_id [fdd90256-0987-4244-bb02-0711ea9639f9] inserido com sucesso!
 986 - task_

# CONSULTA PAGINADA AO DYNAMODB

In [25]:
import boto3
from datetime import datetime

SESSION = boto3.session.Session(region_name='us-east-1', profile_name='sofie')
DYNAMODB = SESSION.client('dynamodb')

LAST_VALUE = {'task_id': {'S': 'b9527993-f1c6-42c1-9f2e-9e60c15e64c3'}}

scan_params = dict(
    TableName='table_micro_task_in_person',
    ProjectionExpression='task_id, address.formatted_address, #status.row_line',
    ExpressionAttributeNames={'#status': 'status'},
    Limit=100
)

if LAST_VALUE:
      scan_params['ExclusiveStartKey'] = LAST_VALUE

dt_start = datetime.now()
response = DYNAMODB.scan(**scan_params)
dt_finish = datetime.now()

qtt = 0
for each in response['Items']:
    print(f'{each["task_id"]["S"]} - {each["address"]["M"]["formatted_address"]["S"]} - Linha: {each["status"]["M"]["row_line"]["N"]}')
    qtt += 1

print(f'\n*** IMPRIMIU-SE {qtt} REGISTROS ***\n*** ÚLTIMO: {response.get("LastEvaluatedKey")} ***')
print(f'*** TEMPO DO SCAN: {(dt_finish - dt_start).microseconds} MICROSEGUNDOS')

0412bcc5-b55e-4ae9-9ff6-c73e2a5f9e90 - Avenida Jabaquara, 947 - Mirandópolis, São Paulo - Linha: 651
91689832-4a1e-4367-94d9-87b511bfae7d - Rua Dorival Seabra, 540 - Baronesa, Osasco - Linha: 353
bee8b19c-3e06-4547-9f8c-38848e2142aa - Rua Sargento Edgard Lourenço Pinto, 127 - Conjunto Habitacional Marechal Mascarenhas de Morais, São Paulo - Linha: 825
81611171-028f-4eb8-a362-374965607300 - Rua João Cordeiro, 370 - Moinho Velho, São Paulo - Linha: 753
1ec09f50-2999-4974-9a7f-25e7bc4b2096 - Rua Juan Vicente, 482 - Bandeiras, Osasco - Linha: 301
b3cccfdd-5602-415a-9d77-7c0f02acde22 - Avenida dos Autonomistas, 896, LOJA 131 - Vila Yara, Osasco - Linha: 460
8bb977a0-df6d-4a0e-89a6-c2f929d3cd9c - Rua Benedito Américo de Oliveira, 2A - Vila Yara, Osasco - Linha: 164
c51de7d6-71b8-4e35-bbdd-b4b824d89e05 - Avenida Maria Campos, 706 - Centro, Osasco - Linha: 236
34a502df-5c6d-4a38-ac5f-ccbc2edf6576 - Rua José Yoshie Yamamoto, 33 - Novo Osasco, Osasco - Linha: 421
6f60a280-1eae-427e-8064-ccbfc3c5

In [2]:
len(DATA['markers'])

339

In [3]:
DATA = {
    "task_id": "a54107a5-a499-4570-b7cd-c16efe79c9e3",
    "execution_id": "8754985749857458947584", 
    
    "task_info": {
        "name": "PAT",
        "version": 1
    },
    
    "location": {
        "lat": -23.5656565656,
        "lng": -43.9564157
    },
        
    "who": "sofier@mysofie.com",
        
    "nps_sofier": {
        "nps": 10,
        "comment": "Tudo lindo e maravilhoso!"
    },
        
    "when": {
        "start": "2019-08-13T20:15:00",
        "finish": "2019-08-13T45:29:00",
    },
      
    "execution": [{
        "context": "LUGAR:ERRADO:LEAD_ALELO",
        "index": 2,
        "previous": 1,
        "when": "2019-08-13T20:15:00",
        "response": "Exemplo",
    }]
}

import json

print(json.dumps(DATA).replace('"', '\\"'))

{\"task_id\": \"a54107a5-a499-4570-b7cd-c16efe79c9e3\", \"execution_id\": \"8754985749857458947584\", \"task_info\": {\"name\": \"PAT\", \"version\": 1}, \"location\": {\"lat\": -23.5656565656, \"lng\": -43.9564157}, \"who\": \"sofier@mysofie.com\", \"nps_sofier\": {\"nps\": 10, \"comment\": \"Tudo lindo e maravilhoso!\"}, \"when\": {\"start\": \"2019-08-13T20:15:00\", \"finish\": \"2019-08-13T45:29:00\"}, \"execution\": [{\"context\": \"LUGAR:ERRADO:LEAD_ALELO\", \"index\": 2, \"previous\": 1, \"when\": \"2019-08-13T20:15:00\", \"response\": \"Exemplo\"}]}
